In [2]:
import pip

pip.main(['install','tabulate','--user'])

0

In [3]:
import os
from tabulate import tabulate
from pprint import pprint
import glob

In [4]:
RESULTS_FOLDER = '/fastdata/cop15rj/results'
SCORES_FOLDER = RESULTS_FOLDER + '/scores'
MODELS_FOLDER = RESULTS_FOLDER + '/models'

In [4]:
os.listdir(RESULTS_FOLDER)

['t-1', 'models', 'scores']

In [5]:
# list size of models

for context_window in [1,2,3,4,5]:
    path = MODELS_FOLDER + '/w'

    path = os.path.join(path, 
                        str(context_window))

    table = []

    for min_count in [5, 10, 20, 50, 100, 200, 300, 400, 500]:
        ipath = os.path.join(path, str(min_count))
    #     print(ipath)

        entry = [min_count, 0, 0, 0, 0]
        
        if not os.path.isdir(ipath):
            continue

        for file in os.listdir(ipath):
            if 'vecs.npy' in file: index = 1
            elif 'contexts.npy' in file: index = 2
            else: continue
            fsize = os.path.getsize(os.path.join(ipath, file))  / (1024**2)
            entry[index] = '%3.2f MB' % (fsize)

        for file in os.listdir(ipath):
            if 'vecs.vocab' in file: index = 3
            elif 'contexts.vocab' in file: index = 4
            else: continue
            vsize = sum(1 for l in open(os.path.join(ipath, file)))
            entry[index] = vsize

        table += [entry,]
    print('context_window:',context_window)
    print(tabulate(table, headers=['min count', 'vecs fsize', 'contexts fsize', 'vecs vsize', 'contexts vsize'],
                  tablefmt='psql'))
    print()

context_window: 1
+-------------+--------------+------------------+--------------+------------------+
|   min count | vecs fsize   | contexts fsize   |   vecs vsize |   contexts vsize |
|-------------+--------------+------------------+--------------+------------------|
|           5 | 962.43 MB    | 5407.22 MB       |       210245 |          1181226 |
|          10 | 598.21 MB    | 3398.17 MB       |       130680 |           742341 |
|          20 | 387.63 MB    | 2177.77 MB       |        84680 |           475742 |
|          50 | 222.40 MB    | 1205.25 MB       |        48585 |           263291 |
|         100 | 147.56 MB    | 757.11 MB        |        32236 |           165394 |
|         200 | 97.25 MB     | 459.43 MB        |        21245 |           100363 |
|         300 | 75.65 MB     | 336.00 MB        |        16527 |            73401 |
|         400 | 63.50 MB     | 265.56 MB        |        13871 |            58013 |
|         500 | 55.18 MB     | 219.92 MB        |        1

In [6]:
path = MODELS_FOLDER + '/m/lst'

table = []

for min_count in [5, 10, 20, 50, 100, 200]:
    ipath = os.path.join(path, 'm-'+str(min_count)+'-mult')
#     print(ipath)
    
    entry = [min_count, 0, 0, 0, 0]

    for file in os.listdir(ipath):
        if 'vecs.npy' in file: index = 1
        elif 'contexts.npy' in file: index = 2
        else: continue
        fsize = os.path.getsize(os.path.join(ipath, file))  / (1024**2)
        entry[index] = '%3.2f MB' % (fsize)
        
    for file in os.listdir(ipath):
        if 'wv' in file: index = 3
        elif 'cv' in file: index = 4
        else: continue
        vsize = sum(1 for l in open(os.path.join(ipath, file), encoding='iso-8859-1'))
        entry[index] = vsize
    
    table += [entry,]
    
print(tabulate(table, headers=['min count', 'vecs fsize', 'contexts fsize', 'vecs vsize', 'contexts vsize']))

  min count  vecs fsize    contexts fsize      vecs vsize    contexts vsize
-----------  ------------  ----------------  ------------  ----------------
          5  0             0                       211380           1181238
         10  0             3398.17 MB              131249            742342
         20  0             2177.77 MB               85039            475742
         50  223.16 MB     1205.25 MB               48750            263291
        100  148.20 MB     757.11 MB                32374            165394
        200  160.36 MB     802.96 MB                35031            175410


In [7]:
print(table)

[[5, 0, 0, 211380, 1181238], [10, 0, '3398.17 MB', 131249, 742342], [20, 0, '2177.77 MB', 85039, 475742], [50, '223.16 MB', '1205.25 MB', 48750, 263291], [100, '148.20 MB', '757.11 MB', 32374, 165394], [200, '160.36 MB', '802.96 MB', 35031, 175410]]


In [8]:
path = SCORES_FOLDER + '/w/' + '*/*/*/*/gap.out'

context_window = 1

scores = {}

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score

# pprint(scores)

In [9]:
file = '/fastdata/cop15rj/results/scores/w/1/5/lst/mult/gap.out'

[float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]

[43.033062532532696]

In [10]:
table = [['', 'LST', 'CIC'], ]

window_size = 1
min_count = 5

rnames = { 'add':'Add', 'avg':'BalAdd', 'mult':'Mult', 'geomean':'BalMult' }

for measure in ['add','avg','mult','geomean']:
    row = [rnames[measure], ]
    for dataset in ['lst','cic']:
        key = (str(win_size), str(min_count), dataset, measure)
        
        if key in scores:
            row.append(scores[key])
        else:
            row.append('-')
    table.append(row)
    
print(tabulate(table, headers="firstrow", tablefmt='psql'))

+---------+---------+--------------------+
|         |     LST | CIC                |
|---------+---------+--------------------|
| Add     | 43.1752 | 40.093547718197925 |
| BalAdd  | 42.2282 | 40.465608353086594 |
| Mult    | 43.0331 | 38.703566722572916 |
| BalMult | 42.971  | -                  |
+---------+---------+--------------------+


In [54]:
path = SCORES_FOLDER + '/w/' + '*/*/*/*/gap.out'

scores = {}

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score

table = []

for win_size in [1,2,3,4,5]:
    for min_count in [5, 10, 20, 50, 100, 200, 300, 400, 500]:
        row = [str(win_size), str(min_count)]
        for dataset in ['lst','cic']:
            for measure in ['add','avg','mult','geomean']:
                key = (str(win_size), str(min_count), dataset, measure)                
                if key in scores:
                    row.append('{:.2f}'.format(scores[key]))
                else:
                    row.append('-')
        
        table.append(row)
    
print(tabulate(table, tablefmt='psql'))

+---+-----+-------+-------+-------+-------+-------+-------+-------+-------+
| 1 |   5 | 43.18 | 42.23 | 43.03 | 42.97 | 40.09 | 40.47 | 38.70 | 39.19 |
| 1 |  10 | 43.67 | 42.61 | 43.69 | 43.35 | 39.60 | 40.04 | 38.32 | 38.84 |
| 1 |  20 | 43.72 | 42.64 | 43.42 | 43.29 | 39.09 | 39.52 | 37.80 | 38.31 |
| 1 |  50 | 43.16 | 41.99 | 43.27 | 42.71 | 37.67 | 38.11 | 36.53 | 37.06 |
| 1 | 100 | 42.19 | 41.07 | 42.39 | 41.87 | 35.98 | 36.39 | 34.92 | 35.42 |
| 1 | 200 | 41.25 | 40.29 | 41.29 | 40.87 | 33.46 | 33.85 | 32.49 | 32.97 |
| 1 | 300 | 40.05 | 39.16 | 40.09 | 39.78 | 31.77 | 32.10 | 30.91 | 31.32 |
| 1 | 400 | 39.09 | 38.31 | 39.38 | 39.06 | 30.51 | 30.87 | 29.71 | 30.14 |
| 1 | 500 | 37.95 | 37.18 | 38.13 | 37.93 | 29.47 | 29.88 | 28.73 | 29.16 |
| 2 |   5 | 40.84 | 40.64 | 40.15 | 40.95 | 39.29 | 39.66 | 38.41 | 38.83 |
| 2 |  10 | 40.98 | 40.61 | 40.24 | 40.96 | 39.08 | 39.39 | 38.20 | 38.64 |
| 2 |  20 | 41.28 | 40.79 | 40.58 | 41.19 | 38.45 | 38.85 | 37.64 | 38.12 |
| 2 |  50 | 

In [44]:
path = SCORES_FOLDER + '/w/' + '*/*/*/*/gap.out'

scores = {}

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score

table = []

min_count = 50
dataset = 'cic'

for measure in ['add','avg','mult','geomean']:
    row = []    
    for win_size in [1,2,3,4,5]:    
        key = (str(win_size), str(min_count), dataset, measure)                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')

    table.append(row)
    
print(tabulate(table, tablefmt='latex'))

\begin{tabular}{rrrrr}
\hline
 37.67 & 37.06 & 36.39 & 36.13 & 36.05 \\
 38.11 & 37.42 & 36.65 & 36.39 & 36.33 \\
 36.53 & 36.33 & 35.76 & 35.59 & 35.56 \\
 37.06 & 36.85 & 36.22 & 35.97 & 35.94 \\
\hline
\end{tabular}


In [41]:
table = []

path = SCORES_FOLDER + '/wn2/' + '*/*/*/*/gap.out'

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
#     print((win_size, min_count, dataset, measure), score)
    scores[(win_size, min_count, dataset, measure)] = score


# for measure in ['mult','geomean','add','avg']:
#     row = [measure, ]
#     for dataset in ['lst','cic']:        
#         key = (str(1), str(50), dataset, measure)                
#         if key in scores:
#             row.append('{:.2f}'.format(scores[key]))
#         else:
#             row.append('-')            
#     table.append(row)

for dataset in ['lst','cic']:
    row = []
    for win in [1, 2, 3, 4, 5]:
        key = (str(win), str(50), dataset, 'none')                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')            
    table.append(row)

print(tabulate(table, tablefmt='latex'))

\begin{tabular}{rrrrr}
\hline
 37.42 & 37.05 & 36.46 & 36.36 & 36.64 \\
 37.29 & 37.08 & 36.35 & 36.09 & 36.13 \\
\hline
\end{tabular}


In [17]:
path = SCORES_FOLDER + '/wn/' + '*/*/*/*/gap.out'

context_window = 1

scores = {}

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score

table = []


for win_size in [1,2,3,4,5]:
    for min_count in [5, 10, 20, 50, 100, 200]:
        row = [str(win_size), str(min_count)]
        for dataset in ['lst','cic']:
            for measure in ['add','baladd','mult','balmult']:
                key = (str(win_size), str(min_count), dataset, measure)                
                if key in scores:
                    row.append('{:.3f}'.format(scores[key]))
                else:
                    row.append('-')
        
        table.append(row)
    
print(tabulate(table, tablefmt='psql'))

+---+-----+--------+--------+--------+--------+--------+--------+--------+--------+
| 1 |   5 | -1.000 | -      | -1.000 | -      | -      | -      | -      | -      |
| 1 |  10 | 42.436 | 40.984 | 43.691 | 43.355 | -      | -      | -      | -      |
| 1 |  20 | 42.312 | 40.879 | 43.425 | 43.292 | -      | -      | -      | -      |
| 1 |  50 | 41.642 | -      | 43.268 | -      | -1.000 | -      | -1.000 | -      |
| 1 | 100 | 40.902 | 39.797 | 42.389 | 41.862 | -1.000 | -      | -1.000 | -      |
| 1 | 200 | 39.698 | 38.697 | 41.291 | 40.867 | -1.000 | -      | -1.000 | -      |
| 2 |   5 | -1.000 | -      | -1.000 | -      | -      | -      | -      | -      |
| 2 |  10 | 40.970 | 39.717 | 40.443 | 40.662 | -1.000 | -1.000 | -1.000 | -1.000 |
| 2 |  20 | 40.996 | 39.552 | 40.587 | 41.150 | -1.000 | -1.000 | -1.000 | -1.000 |
| 2 |  50 | 40.498 | -      | 40.613 | -      | -1.000 | -      | -1.000 | -      |
| 2 | 100 | 39.847 | 38.170 | 39.913 | 40.344 | -1.000 | -1.000 | -1.000 | -

In [19]:
path = SCORES_FOLDER + '/d/' + '*/*/gap.out'

context_window = 1

scores = {}

for file in glob.glob(path):
#     print(file)
    dataset = file.replace(SCORES_FOLDER,'').split('/')[2]
    dim = file.replace(SCORES_FOLDER,'').split('/')[3].split('-')[1]
    measure = file.replace(SCORES_FOLDER,'').split('/')[3].split('-')[-1]
#     print((dataset, dim, measure))
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(dataset, dim, measure)] = score

table = []


for measure in ['mult', 'geomean', 'add', 'avg',]:
    row = []
    for dim in [50, 100, 200, 400, 600, 800, 1000]:
        key = ('lst', str(dim), measure)                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')
        
    table.append(row)
    
print(tabulate(table, tablefmt='latex'))

\begin{tabular}{rrrrrrr}
\hline
 41.49 & 41.99 & 42.59 & 42.38 & 42.51 & 42.52 & 42.51 \\
 41.39 & 41.79 & 42.02 & 41.8  & 42.05 & 41.97 & 41.89 \\
 41.86 & 42.19 & 42.53 & 42.37 & 42.54 & 42.49 & 42.41 \\
 41.19 & 41.4  & 41.44 & 41.25 & 41.39 & 41.3  & 41.21 \\
\hline
\end{tabular}


In [25]:
table = []

for measure in ['add','avg','mult','geomean']:
    row = []
    for min_count in [5, 10, 20, 50, 100, 200]:
        key = (str(1), str(min_count), 'cic', measure)                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')
        
    table.append(row)
    
print(tabulate(table, tablefmt='latex'))

\begin{tabular}{rrrrrr}
\hline
 40.09 & 39.6  & 39.09 & 37.67 & 35.98 & 33.46 \\
 40.47 & 40.04 & 39.52 & 38.11 & 36.39 & 33.85 \\
 38.7  & 38.32 & 37.8  & 36.53 & 34.92 & 32.49 \\
 39.19 & 38.84 & 38.31 & 37.06 & 35.42 & 32.97 \\
\hline
\end{tabular}


In [28]:
path = SCORES_FOLDER + '/n/' + '*/*/gap.out'

context_window = 1

scores = {}

for file in glob.glob(path):
#     print(file)
    dataset = file.replace(SCORES_FOLDER,'').split('/')[2]
    dim = file.replace(SCORES_FOLDER,'').split('/')[3].split('-')[1]
    measure = file.replace(SCORES_FOLDER,'').split('/')[3].split('-')[-1]
#     print((dataset, dim, measure))
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(dataset, dim, measure)] = score

table = []


for measure in ['mult', 'geomean', 'add', 'avg',]:
    row = []
    for dim in [5, 10, 15, 20, 25]:
        key = ('cc', str(dim), measure)                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')
        
    table.append(row)
    
print(tabulate(table, tablefmt='latex', headers=[5, 10, 15, 20, 25]))

\begin{tabular}{rrrrr}
\hline
     5 &    10 &    15 &   20 &    25 \\
\hline
 34.5  & 34.82 & -1    & 34.8 & -1    \\
 -1    & 35.2  & 35.3  & 35.4 & 35.5  \\
 35.42 & -1    & 35.94 & 36   & 36.05 \\
 35.99 & -1    & 36.37 & -1   & 36.5  \\
\hline
\end{tabular}


In [53]:
path = SCORES_FOLDER + '/w/' + '*/*/*/*/*/gap.out'

scores = {}

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-2]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score

path = SCORES_FOLDER + '/wn2/' + '*/*/*/*/gap.out'

for file in glob.glob(path):
    win_size, min_count, dataset, measure = file.replace(SCORES_FOLDER,'').split('/')[2:-1]
    
    score = [float(line.strip().split('\t')[1])*100 for line in open(file) if 'MEAN' in line]
    score = score[0] if len(score)==1 else -1
    
    scores[(win_size, min_count, dataset, measure)] = score


# pprint(scores)
    
table = []
dataset = 'cic'

for measure in ['s-add','s-mult']:
    row = []    
    for win_size in [1,2,3,4,5]:    
        key = (str(win_size), str(50), dataset, measure)                
        if key in scores:
            row.append('{:.2f}'.format(scores[key]))
        else:
            row.append('-')

    table.append(row)
    
row = []    
for win_size in [1,2,3,4,5]:    
    key = (str(win_size), str(100), dataset, 'none')                
    if key in scores:
        row.append('{:.2f}'.format(scores[key]))
    else:
        row.append('-')

table.append(row)
            
print(tabulate(table, tablefmt='latex'))

\begin{tabular}{rrrrr}
\hline
 34.82 & 34.45 & 34.25 & 33.99 & 34.09 \\
 34.95 & 34.43 & 34.38 & 34.07 & 34.09 \\
 35.7  & 35.43 & 34.58 & 34.38 & 34.24 \\
\hline
\end{tabular}
